In [30]:
import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split

In [31]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [32]:
df.shape

(569, 33)

In [33]:
df = df.drop(columns=['id','Unnamed: 32'])

In [34]:
x_train, x_test, y_train, y_test = train_test_split(df.iloc[:,1:], df.iloc[:, 0], test_size=0.2)

In [35]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [36]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

# NumPy arrays to PyTorch tensors

In [37]:
x_train_tensors = torch.from_numpy(x_train)
x_test_tensors = torch.from_numpy(x_test)
y_train_tensors = torch.from_numpy(y_train)
y_test_tensors = torch.from_numpy(y_test)

In [38]:
x_train_tensors.shape

torch.Size([455, 30])

# Defining a model

In [47]:
class SimpleNN():

  def __init__(self, X):
    self.weights = torch.rand(X.shape[1], requires_grad=True, dtype=torch.float64)
    self.bias = torch.zeros(1, requires_grad=True, dtype=torch.float64)

  def forward(self, X):
    z = torch.matmul(X, self.weights) + self.bias
    y_pred = torch.sigmoid(z)
    return y_pred

  def loss_function(self, y_pred, y_true):
    epsilon = 1e-7
    y_pred = torch.clamp(y_pred, epsilon, 1-epsilon)
    loss = -(y_train_tensors * torch.log(y_pred) + (1-y_train_tensors) * torch.log(1-y_pred)).mean()
    return loss

# Defining parameters

In [40]:
learning_rate = 0.1
epochs = 25

# Training pipeline

In [50]:
learning_rate = 0.1
epochs = 25

# create model
model = SimpleNN(x_train_tensors)

# define optimizer (optional, if not using manual updates)
optimizer = torch.optim.SGD([model.weights, model.bias], lr=learning_rate)

# define loop
for epoch in range(epochs):
    # forward pass
    y_pred = model.forward(x_train_tensors)

    # loss calculate
    loss = model.loss_function(y_pred, y_train_tensors)

    # backward pass
    loss.backward()

    # parameters update
    with torch.no_grad():
        model.weights -= learning_rate * model.weights.grad
        model.bias -= learning_rate * model.bias.grad

    # zero gradients
    model.weights.grad.zero_()
    model.bias.grad.zero_()

    # print loss in each epoch
    print(f'Epoch: {epoch + 1}, Loss: {loss.item()}')


Epoch: 1, Loss: 0.5513836061645041
Epoch: 2, Loss: 0.5340323283072835
Epoch: 3, Loss: 0.5172746917935572
Epoch: 4, Loss: 0.501093256810336
Epoch: 5, Loss: 0.48547282835511785
Epoch: 6, Loss: 0.47040051681637296
Epoch: 7, Loss: 0.45586534752215696
Epoch: 8, Loss: 0.44185738334480656
Epoch: 9, Loss: 0.4283664761251176
Epoch: 10, Loss: 0.415380939500333
Epoch: 11, Loss: 0.40288654018311265
Epoch: 12, Loss: 0.3908661403694217
Epoch: 13, Loss: 0.37930009071738924
Epoch: 14, Loss: 0.36816719734320036
Epoch: 15, Loss: 0.3574459244871846
Epoch: 16, Loss: 0.3471155087358541
Epoch: 17, Loss: 0.33658317442283553
Epoch: 18, Loss: 0.3247217635621635
Epoch: 19, Loss: 0.31326978071907124
Epoch: 20, Loss: 0.30221365352722673
Epoch: 21, Loss: 0.2915430515987879
Epoch: 22, Loss: 0.28125068856298824
Epoch: 23, Loss: 0.271332028988712
Epoch: 24, Loss: 0.2617849011296833
Epoch: 25, Loss: 0.25260900265609715


# Evaluation

In [52]:
with torch.no_grad():
    y_pred = model.forward(x_test_tensors)
    y_pred = (y_pred > 0.9).float()
    accuracy = (y_pred == y_test_tensors).float().mean()
    print(f'Accuracy: {accuracy.item()}')

Accuracy: 0.9385964870452881
